In [1]:
### 1: Get mouse coordiantes for hg ee and neg
#awk '{print $1}' ee_dna_seqs.tsv |grep -Ff - lastznet_hg38_all_exons_treated.bed |sed 's/^chr//' > hg38_ee_mm39_coords.bed
#awk '{print $1}' ctrlNeg_dna_seqs.tsv |grep -Ff - lastznet_hg38_all_exons_treated.bed |sed 's/^chr//' > hg38_neg_mm39_coords.bed

### 2: Get fasta sequences of mouse for those coords
#bedtools getfasta -name+ -fi Mus_musculus.GRCm39.dna.primary_assembly.fa -bed hg38_ee_mm39_coords.bed -fo hg38_ee_mm39_seq.fasta
#bedtools getfasta -name+ -fi Mus_musculus.GRCm39.dna.primary_assembly.fa -bed hg38_neg_mm39_coords.bed -fo hg38_neg_mm39_seq.fasta

In [2]:
### Load data
def reverse_complement(seq):
    dna_seq = seq.upper()
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join(complement[base] for base in reversed(dna_seq))

ee_hg_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/ee_dna_seqs.tsv") as file:
    for line in file:            
        if line.strip().split()[0][-1] == "f":
            ee_hg_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            ee_hg_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

neg_hg_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/ctrlNeg_dna_seqs.tsv") as file:
    for line in file:            
        if line.strip().split()[0][-1] == "f":
            neg_hg_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            neg_hg_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

ee_hg_seq_aa={}
with open("/home/mouren/Data/revisions/dn_ds/ee_aa_seqs_hg38.txt") as file:
    for line in file:            
        ee_hg_seq_aa[line.strip().split()[0]] = line.strip().split()[1]

neg_hg_seq_aa={}
with open("/home/mouren/Data/revisions/dn_ds/neg_aa_seqs_hg38.txt") as file:
    for line in file:       
        try:    
            if line.strip().split()[1] != "false":
                neg_hg_seq_aa[line.strip().split()[0]] = line.strip().split()[1]
        except IndexError:
            continue

## Corresponding seqs in mouse 
ee_mm_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/hg38_ee_mm39_seq.fasta") as file:
    id = ""
    for line in file:         
        if line[0] == ">":
            id = (line.strip().split(":")[0]).split(">")[1]
            continue
        else:
            if id[-1] == "f":
                ee_mm_seq_dna[id] = line.strip().split()
            else:
                ee_mm_seq_dna[id] = reverse_complement(line.strip().split()[0])

neg_mm_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/hg38_neg_mm39_seq.fasta") as file:
    id = ""
    for line in file:         
        if line[0] == ">":
            id = (line.strip().split(":")[0]).split(">")[1]
            continue
        else:
            if id[-1] == "f":
                neg_mm_seq_dna[id] = line.strip().split()
            else:
                neg_mm_seq_dna[id] = reverse_complement(line.strip().split()[0])

In [3]:
### Trim dna sequences so that that 
from Bio.Seq import Seq

def map_aa_to_dna(dna_seq, aa_seq):
    dna_seq = dna_seq.upper()
    aa_seq = aa_seq.upper()
    
    for offset in range(3):
        # translate from the current offset
        translated_seq = str(Seq(dna_seq[offset:]).translate(to_stop=False))
        
        # Case 1: Exact match starting from current offset
        if translated_seq.startswith(aa_seq):
            return offset, aa_seq, translated_seq
        
        # Case 2: Skip first AA because incomplete codon (missing bases)
        if translated_seq.startswith(aa_seq[1:]):
            return offset, aa_seq[1:], translated_seq
        
        # Case 3: translated_seq missing first AA, check starting from second AA
        if translated_seq[1:].startswith(aa_seq[1:]):
            return offset + 3, aa_seq[1:], translated_seq[1:]
        
    return None, None, None

ee_hg_seq_dna_trimmed = {}
ee_mm_seq_dna_trimmed = {}
for exon_id, dna_seq in ee_hg_seq_dna.items():
    #check if exon was lifted successfully 
    if exon_id in ee_mm_seq_dna:
        mm_seq = ee_mm_seq_dna[exon_id]
    else:
        continue

    #check if we have aa seq for trimming
    if exon_id in ee_hg_seq_aa:
        aa_seq = ee_hg_seq_aa[exon_id]
    else:
        continue

    offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
    trimmed_dna_ee_hg = dna_seq[offset:]
    trimmed_dna_ee_mm = mm_seq[offset:]

    trimmed_dna_ee_hg_str = ''.join(base for base in trimmed_dna_ee_hg)
    trimmed_dna_ee_mm_str = ''.join(base for base in trimmed_dna_ee_mm)
    

    if trimmed_dna_ee_hg_str != "" and trimmed_dna_ee_mm_str != "" and len(trimmed_dna_ee_hg_str) == len(trimmed_dna_ee_mm_str):
        ee_hg_seq_dna_trimmed[exon_id] = trimmed_dna_ee_hg_str
        ee_mm_seq_dna_trimmed[exon_id] = trimmed_dna_ee_mm_str

neg_hg_seq_dna_trimmed = {}
neg_mm_seq_dna_trimmed = {}
for exon_id, dna_seq in neg_hg_seq_dna.items():
    #check if exon was lifted successfully 
    if exon_id in neg_mm_seq_dna:
        mm_seq = neg_mm_seq_dna[exon_id]
    else:
        continue

    #check if we have aa seq for trimming
    if exon_id in neg_hg_seq_aa:
        aa_seq = neg_hg_seq_aa[exon_id]
    else:
        continue

    offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
    trimmed_dna_neg_hg = dna_seq[offset:]
    trimmed_dna_neg_mm = mm_seq[offset:]

    trimmed_dna_neg_hg_str = ''.join(base for base in trimmed_dna_neg_hg)
    trimmed_dna_neg_mm_str = ''.join(base for base in trimmed_dna_neg_mm)
    

    if trimmed_dna_neg_hg_str != "" and trimmed_dna_neg_mm_str != "" and len(trimmed_dna_neg_hg_str) == len(trimmed_dna_neg_mm_str):
        neg_hg_seq_dna_trimmed[exon_id] = trimmed_dna_neg_hg_str
        neg_mm_seq_dna_trimmed[exon_id] = trimmed_dna_neg_mm_str

/home/mouren/miniconda3/lib/python3.9/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [4]:
from aa import codons  # Import codon table from aa.py

synonymous_count = {}
missense_count = {}

output_file = "/home/mouren/Data/revisions/dn_ds/ee_substitution_results_count_hg_mm.txt"

for exon_id in ee_hg_seq_dna_trimmed:
    hg_seq = ee_hg_seq_dna_trimmed[exon_id]
    mm_seq = ee_mm_seq_dna_trimmed[exon_id]

    syn_count = 0
    mis_count = 0

    for i in range(0, len(hg_seq), 3):
        hg_codon = hg_seq[i:i+3].upper()
        mm_codon = mm_seq[i:i+3].upper()

        if len(hg_codon) != 3 or len(mm_codon) != 3:
            continue  # skip incomplete codons

        # Count base-level differences
        diff_positions = [j for j in range(3) if hg_codon[j] != mm_codon[j]]
        num_differences = len(diff_positions)

        if num_differences == 0:
            continue  # codons are identical

        # Translate both codons
        hg_aa = codons.get(hg_codon, 'X')
        mm_aa = codons.get(mm_codon, 'X')

        if hg_aa == mm_aa:
            syn_count += num_differences
        else:
            mis_count += num_differences

    synonymous_count[exon_id] = syn_count
    missense_count[exon_id] = mis_count

# Save to output file
with open(output_file, 'w') as f:
    f.write("Exon_ID\tSynonymous\tMissense\n")
    for exon_id in synonymous_count:
        syn = synonymous_count[exon_id]
        mis = missense_count[exon_id]
        f.write(f"{exon_id}\t{syn}\t{mis}\n")


In [5]:
### NEG
from aa import codons  # Import codon table from aa.py

synonymous_count = {}
missense_count = {}

output_file = "/home/mouren/Data/revisions/dn_ds/neg_substitution_results_count_hg_mm.txt"

for exon_id in neg_hg_seq_dna_trimmed:
    hg_seq = neg_hg_seq_dna_trimmed[exon_id]
    mm_seq = neg_mm_seq_dna_trimmed[exon_id]

    syn_count = 0
    mis_count = 0

    for i in range(0, len(hg_seq), 3):
        hg_codon = hg_seq[i:i+3].upper()
        mm_codon = mm_seq[i:i+3].upper()

        if len(hg_codon) != 3 or len(mm_codon) != 3:
            continue  # skip incomplete codons

        # Count base-level differences
        diff_positions = [j for j in range(3) if hg_codon[j] != mm_codon[j]]
        num_differences = len(diff_positions)

        if num_differences == 0:
            continue  # codons are identical

        # Translate both codons
        hg_aa = codons.get(hg_codon, 'X')
        mm_aa = codons.get(mm_codon, 'X')

        if hg_aa == mm_aa:
            syn_count += num_differences
        else:
            mis_count += num_differences

    synonymous_count[exon_id] = syn_count
    missense_count[exon_id] = mis_count

# Save to output file
with open(output_file, 'w') as f:
    f.write("Exon_ID\tSynonymous\tMissense\n")
    for exon_id in synonymous_count:
        syn = synonymous_count[exon_id]
        mis = missense_count[exon_id]
        f.write(f"{exon_id}\t{syn}\t{mis}\n")
